In [1]:
import os
import time
import nbimporter

In [2]:
import utils
import blockchain as blockchain_utils
import elliptic_encryption as encryption

Importing Jupyter notebook from utils.ipynb
Importing Jupyter notebook from blockchain.ipynb
Importing Jupyter notebook from elliptic_encryption.ipynb


### LOAD RESOURCES

In [3]:
settings = utils.load_yaml('resources/settings.yaml')

In [4]:
identifier = utils.load_yaml('resources/identifier.yaml')

In [5]:
latest = utils.load_json('resources/latest.json')

### CONNECT TO THE ETHEREUM GATEWAY

In [6]:
web3 = blockchain_utils.connect(settings)

### SERIALIZE THE MANAGER CONTRACT

In [7]:
device_manager = blockchain_utils.contract(latest['devicemanager'], web3, settings)

### HASH THE DEVICE IDENTIFIER

In [8]:
hashed_identifier = utils.hash_data(identifier)

### USE THE HASHED IDENTIFIER TO LOCATE THE DEVICES SMART CONTRACT

In [9]:
device_address = device_manager.read({
    'func': 'fetch_device',
    'params': hashed_identifier
})

### SERIALIZE THE DEVICE SPECIFIC CONTRACT

In [10]:
device_contract = blockchain_utils.contract({
    'address': device_address,
    'abi': latest['device']['abi']
}, web3, settings)

### EVENTS

In [11]:
key_assigned_event = device_contract.event('key_assigned')

### APP PROCESS

In [12]:
try:
    print('TRACKING SYSLOG...\n')
    
    # OPEN THE FILE
    file = open(settings['log_path'], 'r')

    # FIND THE TAILEND OF THE FILE
    file_stats = os.stat(settings['log_path'])
    file_size = file_stats[6]
    
    # GOTO THE LAST ROW
    file.seek(file_size)
    
    # DATA CONTAINERS
    hashes = []
    timestamps = []
    batch = []
    
    # FETCH THE ELLIPTIC ENCRYPTION KEY
    encryption_key = device_contract.read('encryption_key')

    # EVENT LOOP
    while(True):

        # FETCH A NEW ENCRYPTION KEY WHEN ITS UPDATED
        for event in key_assigned_event.get_new_entries():
            encryption_key = device_contract.read('encryption_key')
            print('UPDATED ENCRYPTION KEY!')

        # READ THE LAST LINE
        where = file.tell()
        line = file.readline()

        # IF THE LINE ISNT EMPTY
        if line:

            # EXTRACT THE EVENT PARAMS
            module, code, message, timestamp = utils.parse_line(line)
            
            # HASH THE MODULE & MESSAGE
            hashed = utils.hash_data({
                'module': module,
                'message': message
            })
            
            # ENCODE THE HASH TO BASE64
            #encoded = utils.encode(hashed)

            # PUSH FULL ENTRY TO BATCH CONTAINER
            batch.append({
                'hash': hashed,
                'timestamp': timestamp,
                'module': module,
                'code': code,
                'message': message,
            })

        # IF THE TAILEND OF THE FILE WAS REACHED AND CONTAINER IS NOT EMPTY
        if file.tell() == os.path.getsize(settings['log_path']) and len(batch)>0:
            
            # ENCRYPT THE BATCH
            encrypted = encryption.encrypt(batch, encryption_key)

            # PUSH TO THE CONTRACT
            device_contract.write({
                'func': 'archive',
                'params': [(encrypted, encryption_key)]
            })

            # PRINT MSG
            print('UPLOADED BATCH ({})'.format(len(batch)))

            # RESET BATCH CONTAINER
            batch = []
            
        # SLEEP FOR A SECOND
        time.sleep(1)
        
except:
    print('\nTHE PROCESS WAS MANUALLY KILLED')

TRACKING SYSLOG...

UPLOADED BATCH (12)
UPLOADED BATCH (1)

THE PROCESS WAS MANUALLY KILLED
